In [7]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import importlib
import time

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy import units as u
import h5py
from scipy.optimize import curve_fit

import utils as ut

utils_fold = "C:/Users/Pere/Documents/code/utils/"
sys.path.append(utils_fold)
import mutils as mut
from mutils.plot_utils import Figure
from mutils.plot_utils import Figure3D
importlib.reload(mut)

<module 'mutils' from 'C:\\Users/Pere/Documents/code/utils\\mutils\\__init__.py'>

In [8]:
simulation_name = 'DM-L50-N128'
output_folder = 'output_coarse'
snaps = ut.get_snapshots(f'../data/{simulation_name}/{output_folder}/')
fof_tabs = ut.get_fof_tab(f'../data/{simulation_name}/{output_folder}/')
files_fof = [h5py.File(fof_tab, 'r') for fof_tab in fof_tabs]
files = [h5py.File(snap, 'r') for snap in snaps]
times = [file['Header'].attrs['Time'] for file in files]
zs = [file['Header'].attrs['Redshift'] for file in files]
_, N1 = files[0]['Header'].attrs['NumPart_ThisFile']
M1 = files[0]['Header'].attrs['MassTable'][1] # rest are zero

a = files[0]['Parameters'].attrs['UnitLength_in_cm']
a = a * u.cm
a.to(u.Mpc)

<Quantity 1.00000014 Mpc>

In [9]:
time_steps = [0.0, 0.5, 1.]

J = len(time_steps)
subplots = (1, J)
fig_size = 1920
ratio = 3
ts = 2
Fig = Figure(fig_size=fig_size, subplots=subplots, 
             ratio=ratio, theme='default', 
             wspace=0.025, hspace=0.05, grid=True,
             ts=ts)
fs = Fig.fs
axes = Fig.axes
axes_flat = Fig.axes_flat

resolution = 100
lim = 50

for i, time in enumerate(time_steps):

    ax = axes[0][i]

    idx = np.argmin(np.abs(np.array(times) - time))
    file = files[idx]
    PM = M1 * ut.M_gadget.to(u.Msun).value
    dh = 2*lim/resolution

    time_ = times[idx] * ut.T_gadget.to(u.Gyr).value
    z_ = zs[idx]

    positions = np.array(file['PartType1']['Coordinates']) 

    ranges = [(0, lim), (0, lim), (0, lim)]

    hist, edges = np.histogramdd(positions[:, :], bins=[resolution]*3, range=ranges)
    hist = hist * PM / dh**2

    projection = np.log(np.sum(hist, axis=2))

    ax.imshow((projection.T)[::-1,:], cmap='gnuplot', 
                extent=[-lim, lim, -lim, lim],
                vmin=26, vmax=31
                )
    
    if i == J-1:
        # add colorbar to ax
        cax = ax.inset_axes([1.05, 0.01, 0.075, 0.975])
        cax.tick_params(labelsize=ts*fs,
                        size=0.5*fs, width=0.15*fs,
                        pad=0.1*fs)
        cbar = plt.colorbar(cax=cax, mappable=ax.images[0], orientation='vertical')
        cbar.set_label(r'$\log \sigma$ [M$_\odot$ Mpc$^{-2} h^{2}$]', fontsize=fs*ts)

    if i == 0:
        ax.set_ylabel('y [Mpc $h^{-1}$]', fontsize=fs*ts)
    else:
        ax.tick_params(axis='y', labelleft=False)

    ax.set_xlabel('x [Mpc $h^{-1}$]', fontsize=fs*ts)

    ax.set_title(f'$z={z_:0.2f}, \ t={time_:0.2f}$ Gyr', fontsize=fs*ts,
                    pad=2*fs)

for ax in axes_flat:
    ax.set_facecolor('black')
    ax.grid(color='white',alpha=0.2)

bbox_inches = 'tight'
savepath = f'../figures/{simulation_name}/figure_1.jpg'
Fig.save(savepath, bbox_inches=bbox_inches)
plt.close()

In [131]:
np.array(files_fof[0]['Subhalo']['SubhaloMass'])

array([882.27246 , 212.9272  , 158.4219  , ...,  16.300646,  15.791251,
        11.716089], dtype=float32)

In [5]:
ts = 3
Fig = Figure(fig_size=1920,
             ratio=2, theme='default', 
             wspace=0.025, hspace=0.05, grid=True,
             ts=ts)
fs = Fig.fs
ax = Fig.axes_flat[0]

n_bins = 22

cs = ['k', 'dimgrey', 'darkgrey']
cs2 = ['k', 'dimgrey', 'darkgrey']
fits = []
mins, maxs = [], []
for i in range(3):

    file_fof = files_fof[i]
    NH = file_fof['Header'].attrs['Ngroups_ThisFile']  # number of halos
    NSH = file_fof['Header'].attrs['Nsubhalos_ThisFile']  # number of subhalos
    N_SH_per_H = np.array(file_fof['Group']['GroupNsubs'])  # (array) number of subhalos per halo
    M_per_SH = np.array(file_fof['Subhalo']['SubhaloMass'])  # (array) mass of halos

    M_per_H_units = M_per_SH * M1 * ut.M_gadget.to(u.Msun).value 

    minM = np.min(M_per_H_units)*1.5
    maxM = np.max(M_per_H_units)
                
    print(f'{minM:0.2e}, {maxM:0.2e}')

    lims = np.array([minM, maxM])
    bins = np.logspace(np.log10(minM), np.log10(maxM), n_bins)

    hist, edges = np.histogram(M_per_H_units, bins=bins)

    z_ = zs[i]
    volume = 50**3
    n_distr = hist / volume
    ax.plot(edges[:-1], n_distr, 
           lw=0.5*fs, 
           color=cs[i], 
           label=f'$z={z_:0.0f}$',
           )
    
    phi_star_guess, M_guess, alpha_guess = 1e-2 / volume , 5e10, -1
    popt, pcov = curve_fit(ut.schechter_mf, edges[:-1], n_distr, p0=[phi_star_guess, M_guess, alpha_guess],
                           maxfev=10000
                           )
    fits.append(popt)

    sch_arr = ut.schechter_mf(bins, *popt)
    ax.plot(bins, sch_arr, 
            lw=0.5*fs, 
            color=cs2[i], 
            ls=':',
            )

# set log scale
ax.set_xscale('log')
ax.set_yscale('log')

# set labels
ax.set_xlabel(r'$M_\text{halo} \ [\text{M}_{\! \odot} \, h^{-1}]$', fontsize=fs*ts)
ax.set_ylabel(r'$n$ [$\text{M}_{\! \odot}^{-1} \, \text{Mpc}^{-3} \, h^4 $]', fontsize=fs*ts)

# set legend
ax.legend(fontsize=fs*ts, loc='upper right')

ax.set_xlim(minM, edges[-2])
ax.set_ylim(5e-6, 1e-2)


bbox_inches = 'tight'
savepath = f'../figures/{simulation_name}/figure_2.jpg'
Fig.save(savepath, bbox_inches=bbox_inches)
plt.close()

7.79e+10, 6.80e+12
7.79e+10, 2.47e+13
7.79e+10, 7.34e+13


c:\Users\Pere\Documents\ULL\subjects\q2\tecnicas_simulacion_numerica\gadget4_project\code\utils.py:113: RuntimeWarning: overflow encountered in power
  return phi_star * (M/M_star)**(alpha) * np.exp(-M/M_star)
